# DASHBOARD
__This notebook is meant setup a training session and evaluate its performances__

----

Load python libraries

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import keras
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session, get_session
import keras.losses as klosses
import keras.optimizers as koptimizers
import keras.callbacks as kcallbacks
import keras.initializers as initializers

Using TensorFlow backend.


Load and setup local dependencies

In [2]:
cur_dir = os.getcwd()
base_dir = os.path.dirname(cur_dir)

import sys
sys.path.append(base_dir)
from utils.LungsLoader import LungsLoader
from utils.ScanHandler import ScanHandler

import src.losses as reg_losses
from src.training.config_file import ConfigFile
from src.training.luna_training import LunaTrainer
from src.evaluation.luna_testing import LunaTester

import warnings
warnings.filterwarnings("ignore")

loader = LungsLoader()
handler = ScanHandler(plt)

/Users/dieze/Documents/OMA/TheraPanacea/thera_reg_oma/utils/LungsLoader.py:32: UserWarning: Subset folder 0 not found in data.
  warnings.warn(f"Subset folder {subset_f[-1:]} not found in data.")
/Users/dieze/Documents/OMA/TheraPanacea/thera_reg_oma/utils/LungsLoader.py:32: UserWarning: Subset folder 1 not found in data.
  warnings.warn(f"Subset folder {subset_f[-1:]} not found in data.")
/Users/dieze/Documents/OMA/TheraPanacea/thera_reg_oma/utils/LungsLoader.py:32: UserWarning: Subset folder 2 not found in data.
  warnings.warn(f"Subset folder {subset_f[-1:]} not found in data.")
/Users/dieze/Documents/OMA/TheraPanacea/thera_reg_oma/utils/LungsLoader.py:32: UserWarning: Subset folder 3 not found in data.
  warnings.warn(f"Subset folder {subset_f[-1:]} not found in data.")
/Users/dieze/Documents/OMA/TheraPanacea/thera_reg_oma/utils/LungsLoader.py:32: UserWarning: Subset folder 4 not found in data.
  warnings.warn(f"Subset folder {subset_f[-1:]} not found in data.")
/Users/dieze/Documen

# Initialize a session

In [3]:
session_name = "sandbox_session"
config = ConfigFile(session_name)
config.setup_session()

# Build the model

In [4]:
from src.networks.networks_utils import blocks
from src.networks.MariaNet import MariaNet

__Setting up model hyperparameters :__

In [6]:
input_shape = (64, 64, 64)

conv_block_kwgs = {
                    "activation": "LeakyReLU",
                    "activation_kwargs":{
                        "alpha": 0.3
                        },
                    "normalize": True,
                    "conv_kwargs": {
                      "kernel_size": 3,
                      "padding": "same",
                      "kernel_initializer": initializers.RandomNormal(mean=0.0, stddev=1e-5)
                      }
                    }
squeeze_ratio = 16
conv_block = blocks.ConvBlock(**conv_block_kwgs)
squeeze_block = blocks.SqueezeExciteBlock(ratio=squeeze_ratio)

enc_filters = [32, 64, 128, 32, 32]
enc_dilation = [(1, 1, 1), (1, 1, 1), (2, 2, 2), (3, 3, 3), (5, 5, 5)]
enc_params = [{"filters": n_filter, "dilation_rate": dil_rate} for (n_filter, dil_rate) in zip(enc_filters, enc_dilation)]

dec_filters = [128, 64, 32, 32, 32]
dec_params = [{"filters": n_filter} for n_filter in dec_filters]

def_flow_nf = 3
lin_flow_nf = 12

marianet = MariaNet(input_shape=input_shape,
                    enc_params=enc_params,
                    dec_params=dec_params,
                    conv_block=conv_block,
                    squeeze_block=squeeze_block,
                    def_flow_nf=def_flow_nf,
                    lin_flow_nf=lin_flow_nf)

__Serialize model builder__

In [6]:
builder_path = os.path.join(config.session_dir, "builder.pickle")
marianet.serialize(builder_path)

# Training parameters

__Training, validation, testing sets :__

In [7]:
all_ids = loader.get_scan_ids()
train_ids, val_ids = train_test_split(all_ids, test_size=0.2)
val_ids, test_ids = train_test_split(val_ids, test_size=0.5)

pd.DataFrame(train_ids).to_csv(os.path.join(config.session_dir, LunaTrainer.train_ids_filename), index=False, header=False)
pd.DataFrame(val_ids).to_csv(os.path.join(config.session_dir, LunaTrainer.val_ids_filename), index=False, header=False)
pd.DataFrame(test_ids).to_csv(os.path.join(config.session_dir, LunaTester.test_ids_filename), index=True, header=False)

__Input Shape :__

In [7]:
config.set_input_shape(input_shape)

__Losses :__

In [8]:
losses = [reg_losses.registration_loss, klosses.mean_absolute_error, klosses.mean_absolute_error]
loss_weights = [1., 1e-5, 1e-5]

config.set_losses(losses)
config.set_loss_weights(loss_weights)

__Optimizer :__

In [9]:
optimizer = koptimizers.Adam(lr=1e-2, decay=1e-5)
config.set_optimizer(optimizer)

__Callbacks :__

In [16]:
checkpoints_dir = os.path.join(config.session_dir, ConfigFile.checkpoints_dirname)
tensorboard_dir = os.path.join(config.session_dir, ConfigFile.tensorboard_dirname)


save_callback = kcallbacks.ModelCheckpoint(os.path.join(checkpoints_dir, ConfigFile.checkpoints_format), 
                                           verbose=1, 
                                           save_best_only=True)
early_stopping = kcallbacks.EarlyStopping(monitor='val_loss',
                                          min_delta=1e-3,
                                          patience=20,
                                          mode='auto')
tbCallBack = kcallbacks.TensorBoard(log_dir=tensorboard_dir, 
                                    histogram_freq=0, 
                                    write_graph=True, 
                                    write_images=True)

callbacks = [save_callback, early_stopping, tbCallBack]
config.set_callbacks(callbacks)

__Training scope :__

In [17]:
epochs = 100
steps_per_epoch = 50

config.set_epochs(epochs)
config.set_steps_per_epoch(steps_per_epoch)

__Serialize session configs__

In [18]:
config.serialize()

------------

In [13]:
import utils.IOHandler as io

In [19]:
foo = io.load_pickle(os.path.join(config.session_dir, "config.pickle"))

In [20]:
foo

{'session_name': 'sandbox_session',
 'session_dir': '/Users/dieze/Documents/OMA/TheraPanacea/thera_reg_oma/src/training/../../bin/sandbox_session',
 'input_shape': (64, 64, 64),
 'losses': [<function src.losses.registration_loss(vol1, vol2)>,
  <function keras.losses.mean_absolute_error(y_true, y_pred)>,
  <function keras.losses.mean_absolute_error(y_true, y_pred)>],
 'loss_weights': [1.0, 1e-05, 1e-05],
 'callbacks': [<keras.callbacks.ModelCheckpoint at 0x1a34366550>,
 'metrics': None,
 'epochs': 100,
 'steps_per_epoch': 50,
 'initial_epoch': 0,
 'optimizer_class': keras.optimizers.Adam,
 'optimizer_config': {'lr': 0.009999999776482582,
  'beta_1': 0.8999999761581421,
  'beta_2': 0.9990000128746033,
  'decay': 9.999999747378752e-06,
  'epsilon': 1e-07,
  'amsgrad': False}}